In [5]:
import os
import pandas as pd

cwd = os.getcwd()
path ="/".join(list(cwd.split('/')[0:-1])) 

top_movies = path+'/Data/webscrape/finalmovies.csv'

movies_df = pd.read_csv(top_movies,header=0)

bechdel_data=path+'/Data/bechdel/Bechdel_detailed.csv'
bechdel_df= pd.read_csv(bechdel_data)

# #import titles ds to match the by imdb id 
# title_name = cwd+'/datasets/title.basics.tsv'
# title_name_table= pd.read_csv(title_name, sep='\t', header=0, low_memory=False)


movies_df
bechdel_df.rename(columns={"title":"Title"}, inplace=True)

In [6]:
import re

#rename manually

bechdel_df.loc[bechdel_df['Title'].str.contains('Rogue One'), 'Title'] = bechdel_df['Title'].str.replace('Rogue One', 'Rogue One: A Star Wars Story')

bechdel_df.loc[bechdel_df['Title'].str.contains('Last Jedi'), 'Title'] = bechdel_df['Title'].str.replace('Star Wars: The Last Jedi', 'Star Wars: Episode VIII - The Last Jedi')

bechdel_df.loc[bechdel_df['Title'].str.fullmatch('Star Wars'), 'Title'] = bechdel_df['Title'].str.replace('Star Wars', 'Star Wars: Episode IV - A New Hope')




In [7]:

#remove special characters etc
def normalize_string(s):
    s = s.replace('&#39;','')	
    s = s.replace("'", '')  # apostrophes with empty string
    s = re.sub(r'\W+', '', s)  # Remove non-alphanumeric 
    s = s.lower()  # Convert to lowercase
    s = s.replace(' ', '_') 
    s = s.replace('the', '')# Replace spaces with underscores
    s = s.replace('judgment', 'judgement')
    return s


bechdel_df['name_normalized'] = bechdel_df['Title'].apply(normalize_string)
movies_df['name_normalized'] = movies_df['Title'].apply(normalize_string)

                                                                                                                        
         

In [8]:
final_df = pd.merge(bechdel_df, movies_df, on='name_normalized', how='right')
final_df

,Unnamed: 0,Title_x,year,rating,dubious,imdbid,id,submitterid,date,visible,name_normalized,MOVIE_ID,Title_y,Decade,Genre,Director,Unnamed: 5
0,598.0,Song of the South,1946.0,2.0,0.0,38969.0,5646.0,10566.0,2014-07-22 21:06:00,1.0,songofsouth,0,Song of the South,40s,Animation,"Wilfred Jackson, Harve Foster",NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,samsonanddelilah,1,Samson and Delilah,40s,Historical,Cecil B.DeMille,NaN
2,410.0,Gone with the Wind,1939.0,3.0,NaN,31381.0,291.0,28.0,2009-03-02 14:31:54,1.0,gonewithwind,2,Gone with the Wind,40s,Drama,Victor Fleming,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bellsofstmary,3,The Bells of St. Mary,40s,Musical,Leo McCarey,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sergeantyork,4,Sergeant York,40s,War,Leo McCarey,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,5534.0,Avatar,2009.0,3.0,0.0,499549.0,582.0,312.0,2009-12-22 23:33:32,1.0,avatar,75,Avatar,2010s,SCI-FI,James Cameron,NaN
81,6970.0,Man of Steel,2013.0,3.0,0.0,770828.0,4238.0,7387.0,2013-06-13 06:26:13,1.0,manofsteel,76,Man of Steel,2010s,Action,Zack Snyder,NaN
82,8194.0,Rogue One: A Star Wars Story,2016.0,3.0,0.0,3748528.0,7334.0,14317.0,2016-12-16 20:12:38,1.0,rogueoneastarwarsstory,77,Rogue One: A Star Wars Story,2010s,SCI-FI,Gareth Edwards,NaN
83,6233.0,Harry Potter and the Deathly Hallows: Part 2,2011.0,3.0,1.0,1201607.0,2539.0,3773.0,2011-07-14 20:35:38,1.0,harrypotteranddeathlyhallowspart2,78,Harry Potter and the Deathly Hallows: Part 2,2010s,Fantasy,David Yates,NaN


In [9]:
#study missing values

null_values_x= final_df['Title_x'].isna()
null_values_x.sum()
final_df[null_values_x]

bechdel_no_data= final_df[null_values_x]
bechdel_no_data = bechdel_no_data.drop(["Unnamed: 0","name_normalized"], axis=1)
bechdel_no_data = bechdel_no_data.dropna(axis=1)
bechdel_no_data.rename(columns={"Title_y":"Title"}, inplace= True)
bechdel_no_data.reset_index(drop=True, inplace=True)
bechdel_no_data.to_csv

<bound method NDFrame.to_csv of    MOVIE_ID                  Title Decade       Genre  \
0         1     Samson and Delilah    40s  Historical   
1         3  The Bells of St. Mary    40s     Musical   
2         4          Sergeant York    40s         War   
3         5           Going My Way    40s     Musical   
4         6          Forever Amber    40s       Drama   
5         7    Yankee Doodle Dandy    40s     Musical   
6        18          The Sea Chase    50s         War   
7        29              The Bible    60s  Historical   
8        39    Fiddler on the Roof    70s     Musical   

                        Director  
0                Cecil B.DeMille  
1                    Leo McCarey  
2                    Leo McCarey  
3                   Howard Hawks  
4  John M. Stahl, Otto Preminger  
5                 Michael Curtiz  
6                    John Farrow  
7                    John Huston  
8                 Norman Jewison  >

In [10]:



movies_bechdel = final_df.drop(['Unnamed: 0','submitterid','date','name_normalized', 'id','visible','Title_x'],axis=1)
movies_bechdel.rename(columns={"Title_y":"Title","rating":"bechdel_rating"}, inplace= True)
movies_bechdel = movies_bechdel[['imdbid','Title', 'Decade', 'Genre', 'Director', 'year', 'bechdel_rating', 'dubious']]

title_duplicates = movies_bechdel[movies_bechdel['Title'].duplicated(keep=False)]
# title_duplicates
#drop duplicates by their index
movies_bechdel= movies_bechdel.drop([24,25,29,77])
movies_bechdel.reset_index(drop=True, inplace=True)
# movies_bechdel[movies_bechdel['Title'].duplicated(keep=False)]
movies_bechdel.to_csv(cwd+'/Data/bechdel/all_movies_bechdel.csv')

movies_bechdel


OSError: Cannot save file into a non-existent directory: '/Users/macuser/Desktop/infovizrepo/ProjectGaze/Code/Data/bechdel'

In [ ]:

#no two female characters scene
no_two_females = movies_bechdel[movies_bechdel["bechdel_rating"] == 0.0]
no_two_females.reset_index(drop=True, inplace=True)
no_two_females.to_csv(cwd+'/Data/bechdel/bechdel_failed_0.csv')

#no two females talking
no_talking = movies_bechdel[movies_bechdel["bechdel_rating"] == 1.0]
no_talking.reset_index(drop=True, inplace=True)
no_talking.to_csv(cwd+'/Data/bechdel/bechdel_failed_1.csv')

#two females, but talking about a man

talking_about_male = movies_bechdel[movies_bechdel["bechdel_rating"] == 2.0]
talking_about_male.reset_index(drop=True, inplace=True)
talking_about_male.to_csv(cwd+'/Data/bechdel/bechdel_failed_2.csv')
#passed the bechdel test :)
bechdel_passed = movies_bechdel[movies_bechdel["bechdel_rating"] == 3.0] 
bechdel_passed.reset_index(drop=True, inplace=True)
bechdel_passed.to_csv(cwd+'/Data/bechdel/bechdel_passed.csv')

"No more than one female character:", no_two_females['Title'], "Two female characters, but they do not talk to eachother:", no_talking['Title'], "Two female characters talking about a man:", talking_about_male['Title'], "Passed the bechdel test:", bechdel_passed['Title']
